In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [25]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def reset_parameters(self):
        self.lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        x = self.lin(x)
        return torch.sigmoid(x)

In [26]:
device=0
log_steps=1
num_layers=2
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [27]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [28]:
adj_t = data.adj_t.to(device)

In [29]:
torch.manual_seed(12345)

In [30]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [31]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [32]:
predictor = LinkPredictor(hidden_channels, 1).to(device)

In [33]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [34]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [35]:
model.reset_parameters()
predictor.reset_parameters()

In [36]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [37]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [38]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 0.8652
Epoch: 04, Loss: 0.6364
Epoch: 06, Loss: 0.5625
Epoch: 08, Loss: 0.4445
Epoch: 10, Loss: 0.3779
Epoch: 12, Loss: 0.3376
Epoch: 14, Loss: 0.2986
Epoch: 16, Loss: 0.2701
Epoch: 18, Loss: 0.2463
Epoch: 20, Loss: 0.2251
Epoch: 22, Loss: 0.2089
Epoch: 24, Loss: 0.1937
Epoch: 26, Loss: 0.1781
Epoch: 28, Loss: 0.1666
Epoch: 30, Loss: 0.1562
Epoch: 32, Loss: 0.1506
Epoch: 34, Loss: 0.1402
Epoch: 36, Loss: 0.1340
Epoch: 38, Loss: 0.1295
Epoch: 40, Loss: 0.1251
Epoch: 42, Loss: 0.1210
Epoch: 44, Loss: 0.1174
Epoch: 46, Loss: 0.1142
Epoch: 48, Loss: 0.1107
Epoch: 50, Loss: 0.1080
Epoch: 52, Loss: 0.1048
Epoch: 54, Loss: 0.1020
Epoch: 56, Loss: 0.0997
Epoch: 58, Loss: 0.0974
Epoch: 60, Loss: 0.0964
Epoch: 62, Loss: 0.0935
Epoch: 64, Loss: 0.0919
Epoch: 66, Loss: 0.0893
Epoch: 68, Loss: 0.0879
Epoch: 70, Loss: 0.0865
Epoch: 72, Loss: 0.0853
Epoch: 74, Loss: 0.0836
Epoch: 76, Loss: 0.0822
Epoch: 78, Loss: 0.0811
Epoch: 80, Loss: 0.0799
Epoch: 82, Loss: 0.0784
Epoch: 84, Loss:

In [39]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
    (1): SAGEConv(50, 50)
  )
)

In [40]:
emb.weight

Parameter containing:
tensor([[ 0.5595,  0.3520, -0.2716,  ..., -0.3078, -0.0981, -0.2574],
        [-0.0208,  0.2854, -0.3192,  ...,  0.3020, -0.0811,  0.1794],
        [-0.1035,  0.3032,  0.0214,  ...,  0.2615,  0.0700,  0.0019],
        ...,
        [ 0.0855, -0.5747, -0.0514,  ...,  0.0038,  0.0136,  0.2387],
        [ 0.2376, -0.0240,  0.0927,  ..., -0.0596,  0.2215,  0.0325],
        [ 0.0339, -0.0175, -0.1350,  ...,  0.2698,  0.2447,  0.2858]],
       device='cuda:0', requires_grad=True)

In [41]:
h = model(emb.weight, adj_t)

In [42]:
h

tensor([[ 1.2549,  0.7016, -1.0675,  ..., -1.8488,  1.0179,  0.9163],
        [-2.7863, -1.0170, -0.6418,  ..., -1.6428,  4.5915,  3.7669],
        [-1.0388,  0.7252, -1.0948,  ..., -2.1441,  0.2444,  2.9865],
        ...,
        [ 1.5177, -1.2731,  0.1228,  ..., -1.2389, -0.4281,  2.1538],
        [ 1.0865, -0.8483,  1.4849,  ...,  0.2128,  1.0591,  3.0473],
        [-0.5854, -2.5274, -0.6179,  ...,  0.6285,  1.3054,  5.2778]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [43]:
predictor(h[0], h[411])

tensor([0.9980], device='cuda:0', grad_fn=<SigmoidBackward>)

In [44]:
predictor(h[0], h[2])

tensor([8.1500e-08], device='cuda:0', grad_fn=<SigmoidBackward>)

In [45]:
(h[0] * h[2]).sum()

tensor(40.9070, device='cuda:0', grad_fn=<SumBackward0>)

In [46]:
(h[0] * h[411]).sum()

tensor(162.6263, device='cuda:0', grad_fn=<SumBackward0>)

In [51]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_27_h.npy', h.cpu().detach().numpy())

In [52]:
torch.save(model, '../../models/graphsage_link_pred/27_model.pt')

In [53]:
torch.save(emb, '../../models/graphsage_link_pred/27_embedding.pt')

In [54]:
torch.save(predictor, '../../models/graphsage_link_pred/27_predictor.pt')